In [1]:
from gefs.pc import PC
import pandas as pd
import numpy as np
from experiments.prep import get_stats, normalize_data, standardize_data, learncats, get_dummies
import time
from gefs.trees import RandomForest
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from experiments.prep import get_data

C:\Users\Mennolt van alten\Documents\Data Science - Universiteit\Year 3\BEP\GeFs_Repo\gefs\cluster.py:91: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  ('thr', optional(float64)),
C:\Users\Mennolt van alten\Documents\Data Science - Universiteit\Year 3\BEP\GeFs_Repo\gefs\signed.py:11: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  ('size', int64)
C:\Users\Mennolt van alten\Documents\Data Science - Unive

In [2]:
#general shape of hyperparameter tuner:
#set some default values
#divide data into test and validation/train (resample each time?)
#go into loop:
##fit the system using the hyperparameters
##score it on the validation set using the log_likelyhood
##use the score to determine next choice of hyperparameters (the hard part probably)
##check some kind stopping criterion so it actually stops at some point
#pick the best model you made

#then run it on the test set to see how well we did

In [2]:
#general shape of hyperparameter graph generator:
##get the data
##loop through possible hyperparameter values
##each iteration, split into train and test, make a model then test the model
##save test results into array
##plot array

def hyperparameter_range_test(data, n_cat, values, depth = 5, train_ratio=0.7):
    """
    input:
    Data: 
        some dataset to make a model from
    n_cat: 
        number of categories for each column in the data 
        (get using learncats, for more see experiments.prep.learncats and experiments.prep.get_data)
    Values: hyperparameter values to try (right now trying them on n_estimators)
    depth: what the depth of the tries should be
    train_ratio: ratio of how much data is used for training"""
    
    start = time.time()
    assert train_ratio >= 0
    assert train_ratio <= 1
    shuffle = np.random.choice(range(data.shape[0]), data.shape[0], replace=False)
    data_train = data[shuffle[:int(train_ratio*data.shape[0])], :]
    #splitting test data into dependent and independent variables
    X_train, y_train = data_train[:, :-1], data_train[:, -1] 
    data_test = data[shuffle[int(train_ratio*data.shape[0]):], :]
    #making test data not have a class:
    correct_class_test = data_test[:, -1].copy()
    data_test[:, -1] = np.nan
    print("data split")
    
    performance = []
    accuracy = []
    for i in values:
        #make a new temporary array for saving
        temp_performance = []
        temp_accuracy = [] #accuracy on the original classification
        for j in range(3):
            #redistribute the data
#             shuffle = np.random.choice(range(data.shape[0]), data.shape[0], replace=False)
#             data_train = data[shuffle[:int(train_ratio*data.shape[0])], :]
#             #splitting test data into dependent and independent variables
#             X_train, y_train = data_train[:, :-1], data_train[:, -1] 
#             data_test = data[shuffle[int(train_ratio*data.shape[0]):], :]
#             #making test data not have a class:
#             data_test[:, -1] = np.nan
#             print(f"model {i}, {j} data distributed")
            #make a model
            model = RandomForest(n_estimators=i, ncat=n_cat, max_depth = depth)
            model.fit(X_train, y_train)
            print(f"model {i}, {j} fitted")
            print(int(time.time()-start), "seconds")
            #turn it to a probability circuit
            circuit = model.topc(learnspn=np.Inf) #note: here we can set max_height (default 1000000)
            print(f"model {i}, {j} circuited")
            print(int(time.time()-start), "seconds")
            #test the model
            #return(circuit, data_test, correct_class_test)
            temp_performance.append(circuit.log_likelihood(data_test).mean())
            predictions = circuit.classify(data_test[:, :-1])
            residuals = abs(predictions-correct_class_test)
            cur_accuracy = 1-(sum(residuals)/len(residuals))
            temp_accuracy.append(cur_accuracy)
            print(f"model {i}, {j} tested: acc {cur_accuracy}, log-lik {temp_performance[-1]}")
            print(int(time.time()-start), "seconds")
            #delete model to try again
            circuit.delete()
            model.delete()
            del(model)
        #and save the test results into performance
        performance.append(temp_performance)
        accuracy.append(temp_accuracy)
       
        print(f"{i} finished, {int(time.time()-start)} seconds")
        print(f"Performance: {performance[-1]}")
        
    return(performance, accuracy)
    
    

In [4]:
#circuit, test_data, correct_class_test = hyperparameter_range_test(data, ncat, values)

In [5]:
# predictions = circuit.classify(test_data[:, :-1])
# residuals = predictions-correct_class_test
# cur_accuracy = 1-(sum(residuals)/len(residuals))

In [6]:
#predictions, residuals, cur_accuracy, correct_class_test

In [10]:
#getting the data (taken from getdata function)
# data = "data/dresses.csv"
# data = pd.read_csv(data)
# data = data.replace('?', np.nan)
# data = get_dummies(data)
# data = data.values.astype(float)
# data[data < 0] = np.nan
# ncat = learncats(data)
file_name = "dna.csv"
data, ncat = get_data(file_name, "data/")
depth = 1
#possible hyperparameter values
values = [5*i for i in range(1,141)]

In [40]:
performance, accuracy = hyperparameter_range_test(data, ncat, values, depth)

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 555.08it/s]

data split
model 5, 0 fitted
0 seconds
model 5, 0 circuited
0 seconds
model 5, 0 tested: acc 0.7619047619047619, log-lik -29.969908616918172
0 seconds
model 5, 1 fitted
0 seconds
model 5, 1 circuited
0 seconds
model 5, 1 tested: acc 0.7186147186147186, log-lik -29.818696536664167
0 seconds
model 5, 2 fitted
0 seconds
model 5, 2 circuited
0 seconds
model 5, 2 tested: acc 0.7056277056277056, log-lik -29.881112599393784
0 seconds
5 finished, 0 seconds
Performance: [-29.969908616918172, -29.818696536664167, -29.881112599393784]
model 10, 0 fitted


100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 477.38it/s]


0 seconds
model 10, 0 circuited
0 seconds
model 10, 0 tested: acc 0.7316017316017316, log-lik -29.97040893464387
0 seconds
model 10, 1 fitted
0 seconds
model 10, 1 circuited
0 seconds



100%|█████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 417.40it/s]

...

model 700, 1 circuited
706 seconds
model 700, 1 tested: acc 0.7402597402597403, log-lik -29.752528326128765
707 seconds


  9%|███████▎                                                                        | 64/700 [00:00<00:00, 638.23it/s]

model 700, 2 fitted
708 seconds


100%|███████████████████████████████████████████████████████████████████████████████| 700/700 [00:01<00:00, 389.99it/s]


model 700, 2 circuited
710 seconds
model 700, 2 tested: acc 0.7359307359307359, log-lik -29.47844039056961
710 seconds
700 finished, 711 seconds
Performance: [-29.75337615163951, -29.752528326128765, -29.47844039056961]


In [41]:
# performance_df = pd.DataFrame([values, performance]).transpose()
# performance_df.columns = ["parameter value", "performance (log-likelihood)"]
# performance_df.set_index("parameter value", inplace = True)
# performance_df.plot()
# #very fuzzy results. Idea: run test multiple times with different random divisions.

In [5]:
performance_df = pd.DataFrame(performance, index = values, columns = ["ll1", "ll2", "ll3"])
performance_df['llmean'] = [sum(i)/len(i) for i in performance]
performance_df['values'] = values


acc_df = pd.DataFrame(accuracy, index = values, columns = ["ac1", "ac2", "ac3"])
acc_df['acmean'] = [sum(i)/len(i) for i in accuracy]
acc_df['values'] = values


tested = "number of trees"
fig, ax = plt.subplots(ncols = 2, figsize = (15, 7))
performance_df.plot(kind = 'scatter', x = "values", y = 0, ax = ax[0])
performance_df.plot(kind = 'scatter', x = "values", y = 1, ax = ax[0])
performance_df.plot(kind = 'scatter', x = "values", y = 2, ax = ax[0])
performance_df.plot(kind = 'scatter', x = "values", y = "llmean", color = "orange", title = f"{name}, one train set, depth {depth}", xlabel = tested, ax = ax[0])
ax[0].legend(["0", "1", "2", "mean"])
acc_df.plot(kind = 'scatter', x = "values", y = 0, ax = ax[1])
acc_df.plot(kind = 'scatter', x = "values", y = 1, ax = ax[1])
acc_df.plot(kind = 'scatter', x = "values", y = 2, ax = ax[1])
acc_df.plot(kind = 'scatter', x = "values", y = "acmean", color = "orange", title = f"{name}, one train set, depth {depth}", xlabel = tested, ax = ax[1])
ax[1].legend(["0", "1", "2", "mean"])
#ax.set_xlim(0,200)
#ax.set_ylim(-60.5, -62.5)
cur_moment = str(datetime.datetime.today())[0:19].replace(":", "_")
name = f"{cur_moment}_depth_{depth}_{file_name[0:-4]}"
plt.savefig(f"../graphs/{name}.png")


NameError: name 'performance' is not defined

In [43]:
new_df = acc_df.join(performance_df[performance_df.columns[:-1]])

In [44]:
new_df.to_csv(f"{name}.csv")

In [ ]:
#next thing:
#also make accuracy
#compare accuracy vs. log-likelyhood to see differences
#save into csv for later analysis

#grid search over nr. trees and max depth
#visualize with mean and variance heat maps

In [11]:
def full_test(file_name):
    """
    Does a full test as per the code above for depths 1 to 10
    for file named 'name'
    """
    data, ncat = get_data(file_name, "data/")
    #possible hyperparameter nr trees values
    values = [5*i for i in range(1,11)] #41 default
    start = time.time()
    
    for depth in range(6,11):
        print(depth, "start")
        performance, accuracy = hyperparameter_range_test(data, ncat, values, depth)
        performance_df = pd.DataFrame(performance, index = values, columns = ["ll1", "ll2", "ll3"])
        performance_df['llmean'] = [sum(i)/len(i) for i in performance]
        performance_df['values'] = values


        acc_df = pd.DataFrame(accuracy, index = values, columns = ["ac1", "ac2", "ac3"])
        acc_df['acmean'] = [sum(i)/len(i) for i in accuracy]
        acc_df['values'] = values


        tested = "number of trees"
        fig, ax = plt.subplots(ncols = 2, figsize = (15, 7))
        performance_df.plot(kind = 'scatter', x = "values", y = 0, ax = ax[0])
        performance_df.plot(kind = 'scatter', x = "values", y = 1, ax = ax[0])
        performance_df.plot(kind = 'scatter', x = "values", y = 2, ax = ax[0])
        performance_df.plot(kind = 'scatter', x = "values", y = "llmean", color = "orange", title = f"{file_name}, one train set, depth {depth}", xlabel = tested, ax = ax[0])
        ax[0].legend(["0", "1", "2", "mean"])
        acc_df.plot(kind = 'scatter', x = "values", y = 0, ax = ax[1])
        acc_df.plot(kind = 'scatter', x = "values", y = 1, ax = ax[1])
        acc_df.plot(kind = 'scatter', x = "values", y = 2, ax = ax[1])
        acc_df.plot(kind = 'scatter', x = "values", y = "acmean", color = "orange", title = f"{file_name}, one train set, depth {depth}", xlabel = tested, ax = ax[1])
        ax[1].legend(["0", "1", "2", "mean"])
        #ax.set_xlim(0,200)
        #ax.set_ylim(-60.5, -62.5)
        cur_moment = str(datetime.datetime.today())[0:19].replace(":", "_")
        name = f"{cur_moment}_depth_{depth}_{file_name[0:-4]}"
        plt.savefig(f"../graphs/{name}.png")
        plt.close()
        
        new_df = acc_df.join(performance_df[performance_df.columns[:-1]])
        new_df.to_csv(f"{name}.csv")
        
        end_string = f"/n/ndepth {depth} end: {int(time.time()-start)} seconds /n/n/n"
        print(end_string)

In [12]:
name = "dna.csv"
full_test(name)

6 start
data split


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

model 5, 0 fitted
1 seconds


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:06<00:00,  1.22s/it]


model 5, 0 circuited
7 seconds
model 5, 0 tested: acc 0.8640167364016736, log-lik -98.33882351456441
15 seconds


...

model 50, 1 circuited
4782 seconds
model 50, 1 tested: acc 0.9278242677824268, log-lik -92.49046973661598
4992 seconds


  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

model 50, 2 fitted
5033 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [02:38<00:00,  3.17s/it]


model 50, 2 circuited
5192 seconds
model 50, 2 tested: acc 0.9320083682008369, log-lik -92.94304526548214
5358 seconds
50 finished, 5376 seconds
Performance: [-92.3820042329454, -92.49046973661598, -92.94304526548214]
/n/ndepth 10 end: 14409 seconds /n/n/n
